In [ ]:
# AZA MANAMPY CODE ATO FA MNAOVA CELLULE VAOVAO

from random import randrange
import numpy as np
from sklearn.metrics import mean_squared_error, log_loss
from sklearn.linear_model import HuberRegressor
from sklearn.datasets import load_boston, load_diabetes, load_iris, load_digits
from scipy.special import huber
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

def grad_check_sparse(f, x, analytic_grad, num_checks=12, h=1e-5, error=1e-9):
    """
    sample a few random elements and only return numerical
    in this dimensions
    """

    for i in range(num_checks):
        ix = tuple([randrange(m) for m in x.shape])

        oldval = x[ix]
        x[ix] = oldval + h  # increment by h
        fxph = f(x)  # evaluate f(x + h)
        x[ix] = oldval - h  # increment by h
        fxmh = f(x)  # evaluate f(x - h)
        x[ix] = oldval  # reset

        grad_numerical = (fxph - fxmh) / (2 * h)
        grad_analytic = analytic_grad[ix]
        rel_error = abs(grad_numerical - grad_analytic) / (
            abs(grad_numerical) + abs(grad_analytic)
        )
        print(
            "numerical: %f analytic: %f, relative error: %e"
            % (grad_numerical, grad_analytic, rel_error)
        )
        #assert rel_error < error

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

# Robust linear regression - Huber loss

In [ ]:
data = load_boston()
X_train1, y_train1 = data.data, data.target
w1 = np.random.randn(X_train1.shape[1]) * 0.0001
b1 = np.random.randn(1) * 0.0001

In [ ]:
def vector_product(X, w):
    """ compute a naive (with a for loop) product of the vectors X and w """
    sum = 0.0
    
    for i in range(len(X)):
        sum += X[i] * w[i]
    return sum

def huber_loss(y, w, X, b, epsilon=1.35):
    """ 
    compute the huber loss function 
    
    Inputs:
    -y and b are floats
    -w and X are vectors of the same dimension
    """
    if np.abs(y - vector_product(X,w) - b) <= epsilon:
        return (1.0 / 2.0) * (y - vector_product(X,w) - b)**2
    else:
        return epsilon * (np.abs(y - vector_product(X,w) - b) - 0.5 * epsilon)

def huber_loss_dw(y, w, X, b, i, epsilon=1.35):
    """ 
    compute the derivative of the huber loss function with respect to w 
    
    Inputs:
    -y and b are floats
    -w and X are vectors of the same dimension
    -i index of w
    """
    if np.abs(y - vector_product(X,w) - b) <= epsilon:
        return X[i] * (vector_product(X,w) + b -y)
    else:
        if vector_product(X,w) + b -y <= 0:
            return -epsilon * X[i]
        else:
            return epsilon * X[i]
        
def huber_loss_db(y, w, X, b, epsilon=1.35):
    """ 
    compute the huber loss function derivative with respect to b
    
    Inputs:
    -y and b are floats
    -w and X are vectors of the same dimension
    """
    if np.abs(y - vector_product(X,w) - b) <= epsilon:
        return vector_product(X,w) + b - y
    else:
        if vector_product(X,w) + b -y <= 0:
            return -epsilon 
        else:
            return epsilon

In [ ]:
def huber_loss_naive(w, b, X, y, epsilon=1.35, alpha=0.0001):
    """
    Huber loss for all observations
    
    Inputs:
    - w: array of shape (D,) containing weights
    - b: float bias 
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels 
    - epsilon: float
    - alpha: regularization
    """
    loss = 0.0
    dw = np.zeros_like(w)
    db = 0.0
    
    N = len(X)
    D = len(X[0])
    
    # computation of loss
    for i in range(N):
        loss += huber_loss(y[i], w, X[i], b, epsilon)
    loss = (1 / N) * loss + alpha * (np.linalg.norm(w, ord=2) ** 2)
    
    # computation of dw
    for k in range(D):
        temp = 0.0
        for i in range(N):
            temp += huber_loss_dw(y[i], w, X[i], b, k, epsilon)
        dw[k] = (1 / N) * temp + 2 * alpha * w[k]
        
    # computation of db
    for i in range(N):
        db += huber_loss_db(y[i], w, X[i], b, epsilon)
    db *= (1 / N)
    
    return loss, dw, np.array(db).reshape(1,)

## without regularization

In [ ]:
loss, dw1, db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-9)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-9)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-9)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-9)

 ## with regularization

In [ ]:
loss, dw1, db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-9)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-9)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-9)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-9)

In [ ]:
def huber_loss_vectorized(w, b, X, y, epsilon=1.35, alpha=0.0001):
    """
    Huber loss for all observations
    
    Inputs:
    - w: array of shape (D,) containing weights
    - b: float bias 
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels 
    - epsilon: float
    - alpha: regularization
    """
    loss = 0.0
    dw = np.zeros_like(w)
    db = 0
    
    N = len(X)
    D = len(X[0])
    a = X @ w + b - y
    
    # computation of loss
    h = np.where(np.abs(a) <= epsilon, (1. /2.) * a ** 2, epsilon * (np.abs(a) - 0.5 * epsilon))
    loss = (1 / N) * np.sum(h) + alpha * (np.linalg.norm(w, ord=2) ** 2)
    
    # computation of dw
    h_dw = np.where(np.abs(a) <= epsilon, a, epsilon * np.sign(a))
    dw = (1 / N) * np.sum(h_dw[:, None] * X, axis=0) + 2 * alpha * w

    # computation of db
    h_db = np.where(np.abs(a) <= epsilon, a, epsilon * np.sign(a))
    db = (1 / N) * np.sum(h_db)
    
    return loss, dw, np.array(db).reshape(1,)

## without regularization

In [ ]:
loss, dw1, db1 = huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-9)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-9)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-9)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-9)

## with regularization

In [ ]:
loss, dw1, db1 = huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-9)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-9)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-9)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-9)

In [ ]:
class LinearModel():
    def __init__(self):
        self.w = None
        self.b = None

    def train(self, X, y, learning_rate=1e-3, alpha=0.0001, num_iters=100, batch_size=200, verbose=False):
        N, d = X.shape
        
        if self.w is None: # Initialization
            self.w = 0.001 * np.random.randn(d)
            self.b = 0.0

        # Run stochastic gradient descent to optimize w
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)                                                                                          
            # Hint: Use np.random.choice to generate indices
            i = np.random.choice(N, batch_size)
            X_batch = X[i]
            y_batch = y[i]
            
            # evaluate loss and gradient
            loss, dw, db = self.loss(X_batch, y_batch, alpha)
            loss_history.append(loss)

            # perform parameter update                                                                
            # Update the weights w and bias b using the gradient and the learning rate.          
            self.w = self.w - dw * learning_rate
            self.b = self.b - db * learning_rate
            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))
                
        return loss_history

    def predict(self, X):
        pass

    def loss(self, X_batch, y_batch, reg):
        pass

class HuberRegression(LinearModel):
    """ Linear regression """

    def loss(self, X_batch, y_batch, alpha):
        return huber_loss_vectorized(self.w, self.b, X_batch, y_batch, alpha=alpha)
    
    def predict(self, X):
        return X @ self.w + self.b

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train1)

sk_model = HuberRegressor(fit_intercept=True)
sk_model.fit(X_train1, y_train1)
sk_pred = sk_model.predict(X_train1)
sk_mse = mean_squared_error(sk_pred, y_train1)

model = HuberRegression()
model.train(X_train1, y_train1, num_iters=75000, batch_size=64, learning_rate=1e-2, verbose=True)
pred = model.predict(X_train1)
mse = mean_squared_error(pred, y_train1)

print("MSE scikit-learn:", sk_mse)
print("MSE gradient descent model :", mse)
assert mse - sk_mse < 1

# Multinomial logistic regression

In [ ]:
from scipy.special import softmax 
from scipy.special import logsumexp

In [ ]:
data = load_iris()
X_train2, y_train2 = data.data, data.target

W = np.random.randn(X_train2.shape[1], 3) * 0.0001

Code log-sum-trick pour éviter des problèmes numériques causés par l'exponentiel. 
Source : https://gregorygundersen.com/blog/2020/02/09/log-sum-exp/

In [ ]:

def s(k, W, x):
    """ 
    compute softmax(x)
    
    Inputs:
    - k : int, index of the classes
    - W : array of shape (D, C), Weights
    - x : array of shape (D, ) 
    """
    D = len(x)
    temp = 0
    
    for i in range(D):
        temp += W[i][k] * x[i]
    return temp

def ind(y, k):
    """ indicator function """
    if y == k:
        return 1
    else:
        return 0

In [ ]:
def softmax_loss_naive(W, X, y, alpha):
    """
    Softmax loss function WITH FOR LOOPS

    Inputs:
    - W: array of shape (D, C) containing weights
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - alpha: (float) regularization 

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W;  same shape as W
    """
    
    # Initialization
    
    loss = 0.0
    dW = np.zeros_like(W)
    
    W = np.insert(W, 0, 0, axis=0)
    X = np.insert(X, 0, 1, axis=1)
    N, D = X.shape
    C = len(W[0])
    dW_h = np.zeros((C, D))
    
    # loss computation
    for i in range(N):
        quotient = 0
        temp = 0
        for j in range(C):
            quotient += np.exp(s(j, W, X[i]))
        for k in range(C):
            temp += ind(y[i], k) * np.log(np.exp(s(k, W, X[i])) / quotient )
        loss += temp
    loss *= -(1 / N)
    loss += alpha * np.sum(W ** 2)
    
    # dW computation
    for k in range(C):
        temp2 = 0
        for i in range(N):
            quotient = 0
            for j in range(C):
                quotient += np.exp(s(j, W, X[i]) )
            temp2 += (np.exp(s(k, W, X[i]) / quotient ) - ind(y[i], k)) * X[i]
        dW_h[k] = (1 / N) * temp2   
    dW = np.transpose(dW_h)
    dW += 2 * alpha * W
    
    return loss, dW

## Without regularization

In [ ]:
loss, dW = softmax_loss_naive(W, X_train2, y_train2, 0.0)

f = lambda W: softmax_loss_naive(W, X_train2, y_train2, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

## With regularization

In [ ]:
loss, dW = softmax_loss_naive(W, X_train2, y_train2, 2)

f = lambda W: softmax_loss_naive(W, X_train2, y_train2, 2)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

In [ ]:
def softmax_loss_vectorized(W, X, y, alpha, fit_intercept=False):
    """
    Softmax loss function WITHOUT FOR LOOPS

    Inputs:
    - W: array of shape (D, C) containing weights
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - alpha: (float) regularization 

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W;  same shape as W
    """
    # Initialize the loss and gradient to zero.
    loss = 0.0
    dW = np.zeros_like(W)
                       
    # YOUR CODE HERE
    raise NotImplementedError()

    return loss, dW

## Without regularization

In [ ]:
loss, dW = softmax_loss_vectorized(W, X_train2, y_train2, 0.0)

f = lambda W: softmax_loss_vectorized(W, X_train2, y_train2, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

## With regularization

In [ ]:
loss, dW = softmax_loss_vectorized(W, X_train2, y_train2, 2)

f = lambda W: softmax_loss_vectorized(W, X_train2, y_train2, 2)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

## Gradient descent

In [ ]:
class LinearModel():
    def __init__(self, fit_intercept=True):
        self.W = None
        self.fit_intercept = fit_intercept

    def train(self, X, y, learning_rate=1e-3, alpha=0, num_iters=100, batch_size=200, verbose=False):
        if self.fit_intercept:
            # YOUR CODE HERE
            X = np.insert(X, 0, 1, axis=1)
            
        N, d = X.shape
        
        C = (np.max(y) + 1) 
        if self.W is None: # Initialization
            self.W = 0.001 * np.random.randn(d, C)

        # Run stochastic gradient descent to optimize W
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)                                                                                          
            # Hint: Use np.random.choice to generate indices
            indices = np.random.choice(N, batch_size, replace=False)
            X_batch = X[indices]
            y_batch = y[indices]
            
            # evaluate loss and gradient
            loss, dW = self.loss(X_batch, y_batch, alpha)
            loss_history.append(loss)

            # perform parameter update                                                                
            # Update the weights w using the gradient and the learning rate.          
            self.W -= learning_rate * dW
            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))
                
        return loss_history

    def predict(self, X):
        pass

    def loss(self, X_batch, y_batch, reg):
        pass

class MultinomialLogisticRegressor(LinearModel):
    """ Softmax regression """

    def loss(self, X_batch, y_batch, alpha):
        return softmax_loss_vectorized(self.W, X_batch, y_batch, alpha)
    
    def predict(self, X):
        """ 
        Inputs:
        - X: array of shape (N, D) 

        Returns:
        - y_pred: 1-dimensional array of length N, each element is an integer giving the predicted class 
        """
        if self.fit_intercept:
            X = np.insert(X, 0, 1, axis=1)
        y_pred = X @ self.W
        y_pred = softmax(y_pred, axis=1)
        y_pred = np.argmax(y_pred, axis=1)
        
        return y_pred

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train2 = scaler.fit_transform(X_train2)

sk_model = LogisticRegression(fit_intercept=False)
sk_model.fit(X_train2, y_train2)
sk_pred = sk_model.predict(X_train2)
sk_accuracy = accuracy_score(y_train2, sk_pred)

model = MultinomialLogisticRegressor(fit_intercept=False)
model.train(X_train2, y_train2, num_iters=75000, batch_size=64, learning_rate=1e-3, verbose=True)
pred = model.predict(X_train2)
model_accuracy = accuracy_score(y_train2, pred)

print("Accuracy scikit-learn:", sk_accuracy)
print("Accuracy gradient descent model :", model_accuracy)
assert sk_accuracy - model_accuracy < 0.01

In [ ]:
sk_model = LogisticRegression(fit_intercept=True)
sk_model.fit(X_train2, y_train2)
sk_pred = sk_model.predict(X_train2)
sk_accuracy = accuracy_score(y_train2, sk_pred)

model = MultinomialLogisticRegressor(fit_intercept=True)
model.train(X_train2, y_train2, num_iters=75000, batch_size=64, learning_rate=1e-3, verbose=True)
pred = model.predict(X_train2)
model_accuracy = accuracy_score(y_train2, pred)

print("Accuracy scikit-learn:", sk_accuracy)
print("Accuracy gradient descent model :", model_accuracy)
assert sk_accuracy - model_accuracy < 0.02

# K-Nearest Neighbor

## Computing distances

In [ ]:
data = load_digits()
X_train3, y_train3 = data.data, data.target
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_train3, y_train3, test_size=0.33, random_state=2)

def get_distances_two_loops_with_norm(X_train, X_test):
    num_test = X_test.shape[0]
    num_train = X_train.shape[0]
    distances = np.zeros((num_test, num_train))
    for i in range(num_test):
        for j in range(num_train):
            distances[i, j] = np.linalg.norm(X_test[i] - X_train[j])
    return distances

In [ ]:
def get_distances_two_loops(X_train, X_test):
    """
    Compute the distance between each test point in X_test and each training point
    in X_train 

    Inputs:
    - X_test: array of shape (num_test, D) 

    Returns:
    - distances: array of shape (num_test, num_train), dists[i, j] is Euclidean distance between 
    the ith test point and the jth training point.
    """
    num_test = X_test.shape[0]
    num_train = X_train.shape[0]
    distances = np.zeros((num_test, num_train))
    for i in range(num_test):
        for j in range(num_train):
            # Ataovy ao anaty distances[i, j] ny distance entre ith test point sy jth training point
            # Aza manao boucle instony ato anatiny
            # TSY MAHAZO MAMPIASA np.linalg.norm() :D
            distances[i][j] = np.sqrt(np.sum((X_test[i] - X_train[j])**2))
    return distances

In [ ]:
distances = get_distances_two_loops(X_train3, X_test3)
true_distances = get_distances_two_loops_with_norm(X_train3, X_test3)

difference = np.linalg.norm(distances - true_distances, ord='fro')

print(difference)
assert difference < 1e-10

In [ ]:
def compute_distances_one_loop(X_train, X_test):
    """
    Compute the distance between each test point in X_test and each training point
    in X_train 

    Inputs:
    - X_test: array of shape (num_test, D) 

    Returns:
    - dists: array of shape (num_test, num_train), dists[i, j] is Euclidean distance between 
    the ith test point and the jth training point.
    """
    num_test = X_test.shape[0]
    num_train = X_train.shape[0]
    distances = np.zeros((num_test, num_train))
    temp = np.zeros((num_train, len(X_train[0])))
    for i in range(num_test):
        # Ataovy ao anaty dists[i, j] ny distance entre ith test point sy jth training point
        # Aza manao boucle instony ato anatiny
        # TSY MAHAZO MAMPIASA np.linalg.norm() :D
        temp = np.full((num_train, len(X_train[0])), X_test[i])
        distances[i] = np.transpose(np.sqrt(np.sum((X_train - temp) ** 2, axis=1)))
    return distances    

In [ ]:
distances = compute_distances_one_loop(X_train3, X_test3)
true_distances = get_distances_two_loops_with_norm(X_train3, X_test3)

difference = np.linalg.norm(distances - true_distances, ord='fro')

print(difference)
assert difference < 1e-10

In [ ]:
def get_distances_zero_loop(X_train, X_test):
    """
    Compute the distance between each test point in X_test and each training point
    in X_train 

    Inputs:
    - X_test: array of shape (num_test, D) 

    Returns:
    - distances: array of shape (num_test, num_train), dists[i, j] is Euclidean distance between 
    the ith test point and the jth training point.
    """
    num_test = X_test.shape[0]
    num_train = X_train.shape[0]
    distances = np.zeros((num_test, num_train))  
    D = len(X_train[0])
    temp = np.zeros((num_train, num_test, D))
    temp1 = np.zeros((num_test, num_train, D))
    train_temp = np.zeros((num_test, num_train, D))
    # Ataovy ao anaty dists[i, j] ny distance entre ith test point sy jth training point
    # Aza manao boucle instony
    # TSY MAHAZO MAMPIASA np.linalg.norm() NA FONCTIONS AO AMIN'NY SCIPY :D
    temp = np.full((num_train, num_test, D), X_test)
    temp1 = np.transpose(temp, axes=(1, 0, 2))
    train_temp = np.full((num_test, num_train, D), X_train)
    distances = np.sqrt(np.sum((train_temp - temp1) ** 2, axis=(2)))
    return distances

In [ ]:
distances = get_distances_zero_loop(X_train3, X_test3)
true_distances = get_distances_two_loops_with_norm(X_train3, X_test3)

difference = np.linalg.norm(distances - true_distances, ord='fro')

print(difference)
assert difference < 1e-10

## K-Nearest Neighbor (knn) classifier

In [ ]:
class KNearestNeighborClassifier():
    """ kNN classifier using L2 distance """

    def __init__(self, k=1):
        """
        Inputs:
        - k: number of nearest neighbors that vote for the predicted labels.
        """
        self.k = k

    def fit(self, X, y):
        """
        Train the classifier. Just memorize the training data.

        Inputs:
        - X: array of shape (num_train, D) 
        - y: array of shape (N,) 
        """
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        """
        Predict labels for test data using this classifier.

        Inputs:
        - X: array of shape (num_test, D) 

        Returns:
        - y: array of shape (num_test,) 
        """
        distances = get_distances_zero_loop(self.X_train, X)
        return self.predict_labels(distances)

    def predict_labels(self, distances):
        """
        Given a matrix of distances between test points and training points,
        predict a label for each test point.

        Inputs:
        - distances: array of shape (num_test, num_train), dists[i, j] is Euclidean distance between 
        the ith test point and the jth training point.

        Returns:
        - y:  array of shape (num_test,) 
        """
        num_test = distances.shape[0]
        y_pred = np.zeros(num_test)
        indices = np.zeros(distances.shape[0])
        for i in range(num_test):
            # list storing the labels of the k nearest neighbors to the ith test point.
            closest_y = []

            # Ampidirina ao anaty closest_y ny labels an'ny k neighbors akaiky indrindra
            # Jereo fampiasana np.argsort
            indices = np.argsort(distances[i])
            closest_y = self.y_train[indices[:self.k]]

            # Tadiavo ny label betsaka indrindra dia iny no atao prediction
            # Raha misy mitovy dia izay label kely raisina 
            y_pred[i] = np.argmax(np.bincount(closest_y))
        
        return y_pred

In [ ]:
sk_model = KNeighborsClassifier(n_neighbors=3)
sk_model.fit(X_train3, y_train3)
sk_pred = sk_model.predict(X_test3)
sk_accuracy = accuracy_score(y_test3, sk_pred)

model = KNearestNeighborClassifier(k=3)
model.fit(X_train3, y_train3)
pred = model.predict(X_test3)
model_accuracy = accuracy_score(y_test3, pred)

print("Accuracy scikit-learn:", sk_accuracy)
print("Accuracy gradient descent model :", model_accuracy)
assert sk_accuracy - model_accuracy < 1e-10

## cross-validation

In [ ]:
num_folds = 5
k_choices = [1, 3, 5, 8, 10, 12, 15, 20, 50, 100]

X_train_folds = []
y_train_folds = []

# Split up the data into folds
# X_train_folds and y_train_folds lits of length num_folds

# YOUR CODE HERE
raise NotImplementedError()

# A dictionary of length num_folds holding the accuracies for different values of k 
k_to_accuracies = {}

# Ataovy ary ilay k-fold cross validation 
# Atao ao anaty k_to_accuracies ny accuracy isaky ny valeur k
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
for k in sorted(k_to_accuracies):
    for accuracy in k_to_accuracies[k]:
        print('k = %d, accuracy = %f' % (k, accuracy))